In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt

In [3]:
df_SG = pd.read_excel("US2SG_sample_data.xlsx")

In [12]:
df_ref = pd.read_excel("Traffic_Analysis_20210504.xlsx", sheet_name = 'OTHRAD_ASIN_202104')

In [17]:
df_SG.columns

Index(['gs_arc', 'ASIN', 'gl_desc', 'target_gv_band',
       'target_glance_view_count', 'target_impression_count', 'gv/impression',
       'specific', 'specific (Node ID)', 'brand', 'imp_rank', 'gv_imp_rank',
       'rank', 'size', 'mean', 'median', 'var', 'amax', 'amin',
       'percentile_75', 'percentile_25', 'statue'],
      dtype='object')

In [18]:
df_join = pd.merge(df_ref, df_SG, left_on = 'asin', right_on = 'ASIN', how = 'left')

In [19]:
df_join.columns

Index(['hit_hour_utc', 'session_id', 'request_id', 'attributed_action',
       'seller_ref_marker', 'discovery_ref_marker', 'closer_ref_marker',
       'asin', 'page_type', 'sub_page_type', 'page_type_id',
       'page_type_id_source', 'add_type', 'path_to_from', 'end_time_local',
       'gs_arc', 'ASIN', 'gl_desc', 'target_gv_band',
       'target_glance_view_count', 'target_impression_count', 'gv/impression',
       'specific', 'specific (Node ID)', 'brand', 'imp_rank', 'gv_imp_rank',
       'rank', 'size', 'mean', 'median', 'var', 'amax', 'amin',
       'percentile_75', 'percentile_25', 'statue'],
      dtype='object')

In [20]:
df_result = df_join.reset_index().groupby(['statue','discovery_ref_marker'])['ASIN'].agg({np.size})

In [28]:
df_result = df_result.sort_values(by=['size'], ascending=False)

In [26]:
pd.set_option('display.max_rows', None)

In [30]:
df_result.to_csv("OTHRAD_Match_result.csv", encoding = 'utf-8')

In [ ]:
## this is from other code
for mp in MPs:
    file_path = "C:\\Users\\yingyy\\Desktop\\Discoverability\\Raw_Data\\US-"+mp+"_sort_asin.xlsx"
    df = pd.read_excel(file_path,sheet_name="Sheet1")
    df_brand = pd.read_excel(r"C:\Users\yingyy\Desktop\Discoverability\Top Brands by 2020 GV.xlsx",sheet_name= mp)
    result_1 = df.reset_index().groupby(['gl_desc','brand'])['target_glance_view_count'].agg({np.mean,np.size,np.min,np.max,np.median,np.var}).rename(columns={np.mean:'gv_average',np.size:'size',np.min:'gv_min',np.max:'gv_max',np.median:'gv_median',np.var:'variance'})
    result_1 = result_1.reset_index()
    result_1['brand'] = result_1['brand'].astype(str)
    df_brand['Brand'] = df_brand['Brand'].astype(str)
    result = pd.merge(result_1,df_brand,left_on = 'brand', right_on = 'Brand',how='left')
    result.to_excel(writer, sheet_name = mp)
    result_2 = result[result['size']>=10]
    with open (r"C:\Users\yingyy\Desktop\Discoverability\New Text Document.txt",'a') as file:
        file.write(mp+":"+str(len(pd.unique(result_2['Brand']))))
    result_2 = result_2[['Brand','Rank']].drop_duplicates()
    result_2['Rank_binned'] = pd.cut(result_2['Rank'],bins)
    result_2['Rank_binned'].value_counts(normalize= False,sort = False).to_excel(writer, sheet_name = mp+'_BrandRank')